## LangChainとLangGraphによるRAG・AIエージェント[実践]入門
### 11.2　18のエージェントデザインパターン
- 1. パッシブゴールクリエイター（Passive Goal Creator）
#### passive_goal_creator

与えられた入力や状況から“暗黙の目標”を推定・生成するコンポーネント／設計パターン

入力クエリ => Prompt + LLM で目標を作成 => 次段へ

In [1]:
from importlib.metadata import version
print("langchain-core =", version("langchain-core"))
print("langchain-openai =", version("langchain-openai"))
print("pydantic =", version("pydantic"))

langchain-core = 1.2.2
langchain-openai = 1.1.5
pydantic = 2.12.4


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Goal(BaseModel):
    description: str = Field(..., description="目標の説明")

    @property
    def text(self) -> str:
        return f"{self.description}"


class PassiveGoalCreator:
    def __init__(
        self,
        llm: ChatOpenAI,
    ):
        self.llm = llm

    def run(self, query: str) -> Goal:
        prompt = ChatPromptTemplate.from_template(
            "ユーザーの入力を分析し、明確で実行可能な目標を生成してください。\n"
            "要件:\n"
            "1. 目標は具体的かつ明確であり、実行可能なレベルで詳細化されている必要があります。\n"
            "2. あなたが実行可能な行動は以下の行動だけです。\n"
            "   - インターネットを利用して、目標を達成するための調査を行う。\n"
            "   - ユーザーのためのレポートを生成する。\n"
            "3. 決して2.以外の行動を取ってはいけません。\n"
            "ユーザーの入力: {query}"
        )
        chain = prompt | self.llm.with_structured_output(Goal) # JSON形式で出力
        return chain.invoke({"query": query})


def main():
    import argparse

    from settings import Settings

    settings = Settings()

    parser = argparse.ArgumentParser(
        description="PassiveGoalCreatorを利用して目標を生成します"
    )
    parser.add_argument("--task", type=str, help="実行するタスク", #required=True,
                        default="カレーライスの作り方")   # for Jupyter notebook
    args, _ = parser.parse_known_args()                 # for Jupyter notebook
    #args = parser.parse_args()
    print(f"## imput Task: {args.task}")

    llm = ChatOpenAI(
        model=settings.openai_smart_model, temperature=settings.temperature
    )
    goal_creator = PassiveGoalCreator(llm=llm)
    result: Goal = goal_creator.run(query=args.task)

    print(f"{result.text}")


if __name__ == "__main__":
    main()

## imput Task: カレーライスの作り方
### 目標: カレーライスの作り方を理解し、ユーザーにわかりやすく説明するレポートを作成する。

#### ステップ1: インターネットを利用してカレーライスの作り方を調査する。
- **具体的な行動:**
  - カレーライスの基本的なレシピを検索する。
  - 必要な材料とその分量を確認する。
  - 調理手順を詳細に調べる。
  - 調理時間やコツ、注意点を収集する。

#### ステップ2: 調査結果をもとに、ユーザーのためのレポートを生成する。
- **具体的な行動:**
  - 調査した情報を整理し、わかりやすくまとめる。
  - 材料リストとその分量を明記する。
  - 調理手順をステップごとに詳細に記載する。
  - 調理のコツや注意点を含める。
  - レポートの形式を整え、ユーザーが理解しやすいようにする。

#### 期待される成果:
- ユーザーがカレーライスを作るために必要な情報を網羅したレポートを提供する。
- ユーザーがレポートを参考にして、実際にカレーライスを作ることができるようにする。
